# Data Standardisation

Combining AED and data Wrangling, at the end the csv produced should be a standardised dataset ready for use.

In [2]:
# Import libraries
import sys
import os
import pandas as pd



sys.path.append('../')
os.getcwd()
from core.data_manager import csv_to_dataframe


In [3]:
# Variables
from os import path


path_water = "../data_storage/water.csv"
path_power = "../data_storage/power.csv"
path_gas = "../data_storage/gas.csv"


In [4]:
water_df = csv_to_dataframe(path_water, index_col=False)
gas_df = csv_to_dataframe(path_gas, index_col=False)
power_df = csv_to_dataframe(path_power, index_col=False)

In [5]:
water_df


,date_eom,date,days,id_meter,water_m3,cons,av_day
0,2011-06-30,2011-06-01,0.0,6,1.000,0.000,NaN
1,2011-07-31,2011-07-12,41.0,6,10.000,9.000,0.219512
2,2011-08-31,NaN,NaN,6,NaN,NaN,NaN
3,2011-09-30,NaN,NaN,6,NaN,NaN,NaN
4,2011-10-31,NaN,NaN,6,NaN,NaN,NaN
...,...,...,...,...,...,...,...
184,2024-02-29,2024-02-03,33.0,3,389.949,9.039,0.273909
185,2024-03-31,2024-03-02,28.0,3,397.351,7.402,0.264357
186,2024-04-30,2024-04-07,36.0,3,407.061,9.710,0.269722
187,2024-05-31,2024-05-01,24.0,3,413.556,6.495,0.270625


In [6]:
# Function to sort data based in the id_meter, and then in the date
def sort_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(by=['id_meter', 'date'])
    # if date empty fill with date_eom
    if 'date_eom' in df.columns:
        df['date'] = df['date'].fillna(df['date_eom'])
        df = df.drop(columns=['date_eom'])
    # # drop empty rows in column 3
    # df = df.dropna(subset=[df.columns[3]])
    # convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by=['id_meter', 'date'])
    # day_dif column as the difference between the date and the previous date
    df['days'] = df['date'].diff().dt.days
    # # replace 'days' by 'days_dif' and drop 'days_dif' column
    # df['days'] = df['days'].fillna(df['day_dif'])
    # df = df.drop(columns=['day_dif'])
    # if av_day fill with next value
    df['av_day'] = df['av_day'].bfill()

    return df

water_df = sort_data(water_df)
gas_df = sort_data(gas_df)
power_df = sort_data(power_df)

In [7]:
power_df

,date,days,id_meter,power_kwh,cons,av_day
4,2014-09-04,NaN,4,51419.3,0.0,0.000000
5,2014-12-16,103.0,4,51712.0,292.7,2.841748
6,2015-07-01,197.0,4,52181.7,469.7,2.384264
7,2015-07-23,22.0,4,52182.1,0.4,0.018182
8,2015-09-08,47.0,4,NaN,NaN,2.461429
...,...,...,...,...,...,...
143,2024-06-01,31.0,5,3793.1,109.9,3.545161
0,2011-06-01,-4749.0,7,0.0,0.0,4.068562
1,2012-03-26,299.0,7,1216.5,1216.5,4.068562
2,2014-02-04,680.0,7,2820.2,1349.0,2.358392


In [8]:
# Function resampling independtly for each meter, droping the columns ['av_day', 'days'], and interpolating the data

def resample_data(df: pd.DataFrame, freq: str, name: str) -> pd.DataFrame :
    df_temp = df.copy()
    # split the data in as many dataframes as meters
    df_temp = df_temp.groupby('id_meter')
    # create a list of dataframes with the id_meter as key
    df_temp = [df_temp.get_group(x) for x in df_temp.groups]
    # convert the list of dataframes to independent dataframes
    for i in range(len(df_temp)):
        df_temp[i] = pd.DataFrame(df_temp[i])
    # resample each dataframe
    for i in range(len(df_temp)):
        # drop unnecessary columns ['av_day', 'days', 'water_m3', 'gas_m3', 'power_kwh', or 'cons']
        columns_drop = ['days', 'av_day', 'cons']
        columns_drop = [col for col in columns_drop if col in df_temp[i].columns]
        df_temp[i] = df_temp[i].drop(columns=columns_drop)
        df_temp[i]['date'] = pd.to_datetime(df_temp[i]['date'])
        df_temp[i] = df_temp[i].set_index('date')
        df_temp[i] = df_temp[i].resample(freq).mean()
        df_temp[i] = df_temp[i].interpolate(method='time')
        df_temp[i] = df_temp[i].reset_index()
        # # drop duplicates
        # df_temp[i] = df_temp[i].drop_duplicates(subset=['date'])
        # create a new column [calc_cons] with the difference between the previous and the current value of the third column in the dataframe without using the name of the column
        df_temp[i]['calc_cons'] = df_temp[i].iloc[:, 2].diff() 
        print("Dataframe shape: ", df_temp[i].shape)
    # merge the dataframes
    df_temp = pd.concat(df_temp)
    # sort the data by id_meter and date
    df_temp = df_temp.sort_values(by=['id_meter', 'date'])
    # save the df in a csv file with the name 'resampled_data'+freq+df_name at the data_storage folder
    df_temp.to_csv('../data_storage/resampled_data_'+freq+'_'+name+'.csv', index=False)
    return df_temp

interpolated_water_df = resample_data(water_df, 'D', 'water')
interpolated_gas_df = resample_data(gas_df, 'D', 'gas')
interpolated_power_df = resample_data(power_df, 'D', 'power')


Dataframe shape:  (1994, 4)
Dataframe shape:  (1566, 4)
Dataframe shape:  (1220, 4)
Dataframe shape:  (3559, 4)
Dataframe shape:  (1220, 4)
Dataframe shape:  (2374, 4)
Dataframe shape:  (1186, 4)
Dataframe shape:  (1220, 4)


In [9]:
print(interpolated_water_df)

           date  id_meter    water_m3  calc_cons
0    2014-09-04       2.0  114.418000        NaN
1    2014-09-05       2.0  114.742894   0.324894
2    2014-09-06       2.0  115.067788   0.324894
3    2014-09-07       2.0  115.392682   0.324894
4    2014-09-08       2.0  115.717576   0.324894
...         ...       ...         ...        ...
1215 2014-09-28       6.0  334.125000   0.218750
1216 2014-09-29       6.0  334.343750   0.218750
1217 2014-09-30       6.0  334.562500   0.218750
1218 2014-10-01       6.0  334.781250   0.218750
1219 2014-10-02       6.0  335.000000   0.218750

[4780 rows x 4 columns]
